## Soluções de Sistemas Lineares - Métodos Diretos
### Disciplina de Algoritmos Numéricos - 2021/1
#### Aluno: Thiago Rodrigues Gouveia da Silva





##### Esse notebook tem o intuito de apresentar alguns dos métodos de Soluções de Sistemas Lineares, são eles:
1.  Substituições sucessivas
2.  Substituições retroativas
3.  Eliminação de Gauss
4.  Eliminação de Gauss com pivoteamento
5.  Decomposição LU

In [1741]:
#vetorização
import numpy as np
import copy
import time

A função abaixo tem como intuito resolver o sistema triagular superior **Ux = d** pelas substituições retroativas
Para isso teremos os parâmetros de entrada:

* **n**, ordem da matriz ou números de incógnitas 
* **U**, matriz triangular superior
* **d**, vetor independente

E o parâmetro x será a saída com os valores de x<sub>1</sub>...x<sub>n</sub>

In [1742]:
def backSubstitution(n,U,d):
    x = d/U.diagonal()
    
    for i in range(n-1, -1,-1):
        
        Soma = 0
        
        for j in range(i+1,n,1):
            Soma = Soma + U[i][j] * x[j]
        x[i] = (d[i] - Soma) / U[i][i]
    
    return x

A função abaixo tem como intuito resolver o sistema triagular inferior **Lx = c** pelas substituições sucessivas
Para isso teremos os parâmetros de entrada:

* **n**, ordem da matriz ou números de incógnitas 
* **L**, matriz triangular inferior
* **c**, vetor independente

E o parâmetro x será a saída com os valores de x<sub>1</sub>...x<sub>n</sub>

In [1743]:
def forwardSubstitution(n,L,c):
    #creat output array
    x = np.zeros((n,), dtype='f') 
    #get x1 result
    x[0] = c[0]/L[0][0] 

    j = 0
    for i in range(1, n, 1):
        soma = 0
        for j in range(0, i-1, 1):
            soma = soma + L[i][j] * x[j]
        #get x2...xn
        x[i] = (c[i] - soma) / L[i][j]
        
    return x

A função abaixo tem como intuito resolver o sistema linear por eliminação de gauss.
Para isso teremos os parâmetros de entrada:

 
* **A**, matriz 
* **n**, ordem da matriz ou números de incógnitas
* **b**, vetor independente

Para encontrar o resultado são utilizados os seguintes passos:
1. Encontrar uma matriz triangular superior através da matriz A
2. Utilizar o sistema de substituição retroativo para encontrar os valores de x<sub>1</sub>...x<sub>n</sub>

In [1744]:
def gaussElimination(A,b,n):
    
    #Pivot line
    for k in range(0,n-1,1):
        #Pivot line + 1
        for i in range(k+1, n,1):
            m = (-1) * A[i][k] / A[k][k]
            for j in range (k, n, 1):
                A[i][j] = A[i][j] + m * A[k][j]
            b[i] = b[i] + m * b[k] 
    
    
    return backSubstitution(n, A, b)

A função abaixo tem como intuito resolver o sistema linear por eliminação de gauss.
Para isso teremos os parâmetros de entrada:

 
* **A**, matriz 
* **n**, ordem da matriz ou números de incógnitas
* **b**, vetor independente

Para encontrar o resultado são utilizados os seguintes passos:
1. Encotrar o maior valor em uma determinada coluna na matriz A e inverter sua linha com a linha pivô daquela coluna 
2. Zerar os valores abaixo desse pivô utilizando Gauss
3. Repetir esses passos até que seja encontrada uma matriz triangular superior
4. Utilizar o sistema de substituição retroativo para encontrar os valores de x<sub>1</sub>...x<sub>n</sub>


In [1745]:
def gaussPivot(A,b,n):
    
    for k in range(0,n-1,1):
        #find pivot
        maior = abs(A[k][k])
        for i in range(k+1, n, 1):
            if abs(A[i][k]) > maior:
                maior = abs(A[i][k])
                r = i
            else: 
                r = k
        #line switching
        for i in range(0, n, 1):
            v = A[k][i]
            A[k][i] = A[r][i]
            A[r][i] = v 
        
        v = b[k]
        b[k] = b[r]
        b[r] = v
        
        #regular Gauss elimination
        for i in range(k+1, n, 1):
            
            m = (-1) * A[i][k] / A[k][k]
            for j in range (k, n, 1):
                A[i][j] = A[i][j] + m * A[k][j] 
                
                
            b[i] = b[i] + m * b[k]
                  
    return backSubstitution(n, A, b)

A função abaixo tem como intuito resolver o sistema linear por eliminação de gauss.
Para isso teremos os parâmetros de entrada:

 
* **A**, matriz 
* **n**, ordem da matriz ou números de incógnitas
* **b**, vetor independente

Para encontrar o resultado são utilizados os seguintes passos:

1. Criar a matriz identidade
2. Decompor a matriz A
3. Extrair U e L de A
4. Multiplica P e b
5. Calcula Ly = Pb ->  y = L<sup>-1</sup> * Pb
6. Calcula Ux = y ->  x = U<sup>-1</sup> * y

In [1746]:
def gaussLU(A,b,n):
    # identity array
    P = np.identity(n)
    
    for j in range(0, n-1, 1):
        p = j
        max = A[j][j]
        for k in range(j+1, n, 1):
            if abs(A[k][j]) > max:
                max = abs(A[k][j])
                p = k
        if(j != p):
            for k in range(0, n, 1):
                t = A[j][k]
                A[j][k] = A[p][k]
                A[p][k] = t
            
            t = copy.deepcopy(P[j])
            P[j] = copy.deepcopy(P[p])
            P[p] = copy.deepcopy(t)
        
        
        if abs(A[j][j]) != 0:
            for i in range(j+1, n, 1):
                m = A[i][j] / A[j][j]
                A[i][j] = m
                for k in range(j+1, n, 1):
                    A[i][k] = A[i][k] - m * A[j][k]
    
    #Extracting L from A
    L = np.tril(A)
    #Diagonal fill with 1's
    np.fill_diagonal(L, 1)
    #Extracting U from A
    U = np.triu(A)
    
    #mult P * b
    Pb = np.matmul(P, b)
    
    #mult L^-1 * Pb
    y = np.matmul(np.linalg.inv(L), Pb)
    #mult U^-1 * Pb
    x = np.matmul(np.linalg.inv(U), y)

    return x       


A função abaixo tem como intuito resolver o sistema linear usando o Algoritmo de Jacobi.
Para isso teremos os parâmetros de entrada:

 
* **A**, matriz 
* **n**, ordem da matriz ou números de incógnitas
* **b**, vetor independente
* **Toler**, tolerância
* **iterMax**, número máximo de iterações




In [1747]:
def Jacobi(n, A, b, Toler, iterMax):
    #creat output array
    x = np.zeros((n,), dtype='f')
    v = np.zeros((n,), dtype='f') 
    
    timeInit = time.time()
    
    for i in range(n):
        x[i] = b[i] / A[i][i]
    
    Iter = 0
    
    while True:
        Iter = Iter + 1
        for i in range(n):
            Sum = 0
            for j in range(n):
                if i != j:
                    Sum = Sum + A[i][j] * x[j]
            v[i] = (b[i] - Sum) / A[i][i]
    
        normaNum = 0
        normaDen = 0
    
        for i in range(n):
        
            t = abs(v[i] - x[i])
        
            if t > normaNum:
                normaNum = t
            if abs(v[i]) > normaDen:
                normaDen = abs(v[i])
            
            x[i] = v[i]
    
        normaRel = normaNum / normaDen
        
        print("Numero de iterações: ", Iter)
        print("Norma Relativa: ", normaRel)
        print("Vet: ", x)
        
        if normaRel <= Toler or Iter >= iterMax:
            break
    
    if normaRel <= Toler:
        Info = 0
        print("Convergiu")
    else:
        Info = 1
        print("Não convergiu")
    
    print("Tempo de execução: ", time.time() - timeInit)
    
    return x  

A função abaixo tem como intuito resolver o sistema linear usando o Algoritmo de Gauss-Seidel.
Para isso teremos os parâmetros de entrada:

 
* **A**, matriz 
* **n**, ordem da matriz ou números de incógnitas
* **b**, vetor independente
* **Toler**, tolerância
* **iterMax**, número máximo de iterações

In [1748]:
def gaussSeidel(n, A, b, Toler, iterMax):
    #creat output array
    x = np.zeros((n,), dtype='f')
    v = np.zeros((n,), dtype='f') 
    
    timeInit = time.time()

    for i in range(n):
        x[i] = b[i] / A[i][i]
    
    Iter = 0
    
    while True:
        Iter = Iter + 1
        normaNum = 0
        normaDen = 0
        for i in range(n):
            Sum = 0
            for j in range(n):
                if i != j:
                    Sum = Sum + A[i][j] * x[j]
            
            v[i] = x[i]
            x[i] = (b[i] - Sum) / A[i][i]
            t = abs(v[i] - x[i])
            
            if t > normaNum:
                normaNum = t
            if abs(x[i]) > normaDen:
                normaDen = abs(x[i])

    
        normaRel = normaNum / normaDen
        
        print("Numero de iterações: ", Iter)
        print("Norma Relativa: ", normaRel)
        print("Vet: ", x)
        
        if normaRel <= Toler or Iter >= iterMax:
            break
    
    if normaRel <= Toler:
        Info = 0
        print("Convergiu")
    else:
        Info = 1
        print("Não convergiu")
    
    print("Tempo de execução: ", time.time() - timeInit)
    
    return x  

Criação da matriz A e do vetor independente b, com o auxílio da biblioteca numpy

In [1749]:
n = 3  
A = np.array([[1, -3, 2], [-2, 8, -1], [4, -6, 5]], dtype='f')
b = np.array([11, -15, 29], dtype='f')

Resultado da eliminação de Gauss utilizando a os valores inicializados acima

In [1750]:
res = gaussElimination(A,b,n)
print(res)

[ 2. -1.  3.]


Resultado da eliminação de Gauss com pivoteamento utilizando a os valores inicializados acima

In [1751]:
res = gaussPivot(A,b,n)
print(res)

[ 2. -1.  3.]


Resultado da Decomposição LU utilizando a os valores inicializados acima

In [1752]:
res = gaussLU(A,b,n)
print(res)

[ 1.99999928 -1.          3.00000009]


Resultado do Algoritmo de Jacobini utilizando os valores inicializados acima

In [1753]:
res = Jacobi(n,A,b,0,50)

Numero de iterações:  1
Norma Relativa:  0.29032257
Vet:  [15.5 -1.   3. ]
Numero de iterações:  2
Norma Relativa:  4.5
Vet:  [ 2. -1.  3.]
Numero de iterações:  3
Norma Relativa:  0.0
Vet:  [ 2. -1.  3.]
Convergiu
Tempo de execução:  0.0009999275207519531


Resultado do Algoritmo de Gauss-Seidel utilizando os valores inicializados acima

In [1754]:
res = gaussSeidel(n,A,b,0,50)

Numero de iterações:  1
Norma Relativa:  0.29032257
Vet:  [15.5 -1.   3. ]
Numero de iterações:  2
Norma Relativa:  4.5
Vet:  [ 2. -1.  3.]
Numero de iterações:  3
Norma Relativa:  0.0
Vet:  [ 2. -1.  3.]
Convergiu
Tempo de execução:  0.0010004043579101562
